<a href="https://colab.research.google.com/github/dianagromakovskaya/stock-markets-analytics-zoomcamp/blob/main/homework/homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install yfinance

In [3]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

# Question 1: IPO Filings Web Scraping and Data Processing
**What's the total sum ($m) of 2023 filings that happened on Fridays?**

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/ Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs). Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given). You may be inspired by the function extract_numbers() in [Code Snippet 4], or you can write your own function to "parse" a string. Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

Find the total sum in $m (millions of USD, closest INTEGER number) for all filings during 2023, which happened on Fridays (Date.dt.dayofweek()==4). You should see 32 records in total, 25 of it is not null.

(additional: you can read about S-1 IPO filing to understand the context)

In [4]:
import pandas as pd
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings"
response = requests.get(url, headers=headers)

ipo_df = pd.read_html(response.text)[0]

In [5]:
ipo_df.head(100)

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"May 3, 2024",TBN,Tamboran Resources Corporation,-,-
1,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000
2,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
3,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
4,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-
...,...,...,...,...,...
95,"Sep 29, 2023",VAPA,Valens Pay Global Limited,$5.00 - $6.00,1000000
96,"Sep 28, 2023",TKE,OUI Global,$4.00 - $6.00,2500000
97,"Sep 26, 2023",MTRS,"Metros Development Co., Ltd.",$8.00 - $9.00,1000000
98,"Sep 25, 2023",BTOC,Armlogi Holding Corp.,$5.00 - $6.00,2000000


In [6]:
ipo_df['Filing Date'] = pd.to_datetime(ipo_df['Filing Date'], format='%b %d, %Y')

In [7]:
ipo_df['Shares Offered'] = pd.to_numeric(ipo_df['Shares Offered'], errors='coerce')
ipo_df['Shares Offered'] = ipo_df['Shares Offered'].astype(float)

In [8]:
ipo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324 entries, 0 to 323
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Filing Date     324 non-null    datetime64[ns]
 1   Symbol          324 non-null    object        
 2   Company Name    324 non-null    object        
 3   Price Range     324 non-null    object        
 4   Shares Offered  250 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 12.8+ KB


In [9]:
import re


def get_avg_price(price_range):
    if price_range == '-':
      return None
    else:
      price = re.findall(r'\$(\d+).(\d+)', price_range)
      if len(price) == 1:
        return int(price[0][0]) + int(price[0][1]) / 100
      elif len(price) == 2:
        return (int(price[0][0]) + int(price[0][1]) / 100 + int(price[1][0]) + int(price[1][1]) / 100) / 2

In [10]:
ipo_df['avg_price'] = ipo_df['Price Range'].apply(lambda x: get_avg_price(x))

In [11]:
ipo_df.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,avg_price
0,2024-05-03,TBN,Tamboran Resources Corporation,-,NaN,NaN
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,3.0
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,10.0
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,10.0
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN,NaN


In [12]:
ipo_df['shares_offered_value'] = ipo_df['Shares Offered'] * ipo_df['avg_price']

In [13]:
filtered_ipo_df = ipo_df[(ipo_df['Filing Date'].dt.year == 2023) & (ipo_df['Filing Date'].dt.dayofweek == 4)]

In [14]:
filtered_ipo_df['shares_offered_value'].sum() / 10**6

285.7

# Question 2: IPOs "Fixed days hold" strategy
**Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?**

Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/). Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance.

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks. Also, you can see the ticker changes using this link. Some of the tickers (like 'DYCQ' and 'LEGT') were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day). Find X, when the 75% quantile growth (among 185 investments) is the highest.

HINTs:

You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
You can use the DataFrame.describe() function to get mean, min, max, 25-50-75% quantiles.
Additional:

You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
What's your recommendation: Do you suggest pursuing this strategy for an optimal X?

In [15]:
url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)

ipos_2023 = pd.read_html(response.text)[0]

In [16]:
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipos_2024 = pd.read_html(response.text)[0]

In [17]:
stacked_ipos_df = pd.concat([ipos_2024, ipos_2023], ignore_index=True)

In [18]:
stacked_ipos_df.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"May 9, 2024",SVCO,"Silvaco Group, Inc.",$19.00,$19.80,7.68%
1,"May 9, 2024",PAL,"Proficient Auto Logistics, Inc.",$15.00,$14.75,-0.90%
2,"May 8, 2024",NNE,Nano Nuclear Energy Inc.,$4.00,$5.05,34.75%
3,"May 1, 2024",VIK,Viking Holdings Ltd.,$24.00,$28.50,18.75%
4,"Apr 26, 2024",ZONE,"CleanCore Solutions, Inc.",$4.00,$3.17,-20.75%


In [19]:
stacked_ipos_df['IPO Date'] = pd.to_datetime(stacked_ipos_df['IPO Date'], format='%b %d, %Y')

In [20]:
tickers = list(stacked_ipos_df[stacked_ipos_df['IPO Date'] < '2024-03-01']['Symbol'])
tickers.remove('RYZB')


In [21]:
len(tickers) == 184

True

In [22]:
url = "https://stockanalysis.com/actions/changes/"
response = requests.get(url, headers=headers)

ticker_changes = pd.read_html(response.text)[0]

In [23]:
ticker_changes.head()

,Date,Old,New,New Company Name
0,"May 3, 2024",CONX,CNXX,Conx Corp
1,"Apr 30, 2024",FUV,FUVV,Arcimoto Inc
2,"Apr 19, 2024",XLYO,XYLO,Xylo Technologies Ltd.
3,"Apr 19, 2024",DCFCW,DCFWQ,Tritium Dcfc Ltd.
4,"Apr 18, 2024",MDGS,XYLO,Xylo Technologies Ltd.


In [24]:
ticker_changes[ticker_changes['Old'].isin(tickers)]

,Date,Old,New,New Company Name
81,"Jan 16, 2024",PTHR,HOVR,New Horizon Aircraft Ltd.
206,"Jul 20, 2023",CLBR,PSQH,Psq Holdings Inc


In [25]:
new_names = {'PTHR': 'HOVR', 'CLBR': 'PSQH', 'IBAC': 'IBACU'}

In [ ]:
import time

stocks_df = pd.DataFrame({'A' : []})

for i, ticker in enumerate(tickers):
  if ticker in new_names:
    ticker = new_names[ticker]
  print()
  print(i, ticker)

  # Work with stock prices
  history_prices = yf.download(tickers=ticker,
                               period="max",
                               interval="1d")
  history_prices['Ticker'] = ticker
  history_prices['Year']= history_prices.index.year
  history_prices['Month'] = history_prices.index.month
  history_prices['Weekday'] = history_prices.index.weekday
  history_prices['Date'] = history_prices.index.date

  for i in range(1, 31):
    history_prices['growth_future' + str(i) + 'd'] = history_prices['Adj Close'].shift(-i) / history_prices['Adj Close']

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = history_prices
  else:
    stocks_df = pd.concat([stocks_df, history_prices], ignore_index=True)

[*********************100%%**********************]  1 of 1 completed


0 SMXT



[*********************100%%**********************]  1 of 1 completed


1 VHAI



[*********************100%%**********************]  1 of 1 completed

...


181 SKWD



[*********************100%%**********************]  1 of 1 completed


182 ISRL



[*********************100%%**********************]  1 of 1 completed


183 MGOL


In [ ]:
stocks_df.head()

,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_future21d,growth_future22d,growth_future23d,growth_future24d,growth_future25d,growth_future26d,growth_future27d,growth_future28d,growth_future29d,growth_future30d
0,3.50,9.35,3.500,8.00,8.00,2840400,SMXT,2024,2,1,...,0.987500,1.143750,1.370000,1.342500,1.648750,1.441250,1.412500,1.420000,1.545000,1.485000
1,6.42,7.24,5.600,5.86,5.86,243200,SMXT,2024,2,2,...,1.561433,1.870307,1.832764,2.250853,1.967577,1.928328,1.938566,2.109215,2.027304,2.329351
2,5.74,5.84,4.300,4.89,4.89,130000,SMXT,2024,2,3,...,2.241309,2.196319,2.697342,2.357873,2.310839,2.323108,2.527607,2.429448,2.791411,2.750511
3,3.94,5.15,3.800,4.88,4.88,485000,SMXT,2024,3,4,...,2.200820,2.702869,2.362705,2.315574,2.327869,2.532787,2.434426,2.797131,2.756147,3.055328
4,4.73,5.10,4.258,4.66,4.66,114300,SMXT,2024,3,0,...,2.830472,2.474249,2.424893,2.437768,2.652361,2.549356,2.929185,2.886266,3.199571,2.605150


In [ ]:
min_dates = stocks_df.groupby(by='Ticker').agg({'Date': 'min'})

In [ ]:
growth_future_after_ipo_df = pd.merge(stocks_df, min_dates, how='inner', on=['Date', 'Ticker'])

In [ ]:
growth_future_after_ipo_df.head()

,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,...,growth_future21d,growth_future22d,growth_future23d,growth_future24d,growth_future25d,growth_future26d,growth_future27d,growth_future28d,growth_future29d,growth_future30d
0,3.50,9.35,3.50,8.000,8.000,2840400,SMXT,2024,2,1,...,0.987500,1.143750,1.370000,1.342500,1.648750,1.441250,1.412500,1.420000,1.545000,1.485000
1,1.39,1.80,1.13,1.635,1.635,2444600,VHAI,2024,2,0,...,0.538838,0.500917,0.488073,0.428135,0.366972,0.305199,0.274006,0.242813,0.229358,0.243425
2,10.09,10.11,10.07,10.100,10.100,454200,DYCQ,2024,4,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6.00,6.00,4.54,4.800,4.800,357000,CHRO,2024,2,4,...,0.633333,0.595833,0.579167,0.512500,0.520833,0.508333,0.564583,0.545833,0.497917,0.500000
4,4.00,4.40,2.50,3.030,3.030,373200,UMAC,2024,2,2,...,0.795380,0.798680,0.646865,0.577558,0.590759,0.551485,0.594059,0.669967,0.775578,0.577558


In [ ]:
growth_future_after_ipo_df.describe().loc['75%']

Open                1.317500e+01
High                1.545250e+01
Low                 1.252500e+01
Close               1.326500e+01
Adj Close           1.309393e+01
Volume              3.381900e+06
Year                2.023000e+03
Month               9.000000e+00
Weekday             4.000000e+00
growth_future1d     1.012352e+00
growth_future2d     1.018409e+00
growth_future3d     1.009551e+00
growth_future4d     1.008924e+00
growth_future5d     1.009369e+00
growth_future6d     1.007147e+00
growth_future7d     1.006871e+00
growth_future8d     1.008127e+00
growth_future9d     1.011545e+00
growth_future10d    1.010670e+00
growth_future11d    1.012795e+00
growth_future12d    1.016016e+00
growth_future13d    1.019795e+00
growth_future14d    1.016382e+00
growth_future15d    1.016284e+00
growth_future16d    1.014190e+00
growth_future17d    1.010843e+00
growth_future18d    1.013940e+00
growth_future19d    1.014946e+00
growth_future20d    1.014496e+00
growth_future21d    1.012850e+00
growth_fut

# Question 3: Is Growth Concentrated in the Largest Stocks?
**Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?**

Reuse [Code Snippet 5] to obtain OHLCV stats for 33 stocks for 10 full years of data (2014-01-01 to 2023-12-31). You'll need to download slightly more data (7 periods before 2014-01-01 to calculate the growth_7d for the first 6 days correctly):

`US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']`

`EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']`

`INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']`

`LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS`

Now let's add the top 12-22 stocks (as of end-April 2024):

`NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']`

`NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']`

`NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']`

`LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA`

You should be able to obtain stats for 33 LARGEST STOCKS and 32 LARGE STOCKS (from the actual stats on Yahoo Finance)

Calculate growth_7d for every stock and every day. Get the average daily growth_7d for the LARGEST_STOCKS group vs. the LARGE_STOCKS group.

For example, for the first of data you should have:

Date	ticker_category	growth_7d
2014-01-01	LARGE	1.011684
2014-01-01	LARGEST	1.011797
On that day, the LARGEST group was growing faster than LARGE one (new stocks).

Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP, divide it by the total number of trading days (which should be 2595 days), and convert it to a percentage (closest INTEGER value). For example, if you find that 1700 out of 2595 days meet this condition, it means that 1700/2595 = 0.655, or approximately 66% of days, the LARGE stocks were growing faster than the LARGEST ones. This suggests that you should consider extending your dataset with more stocks to seek higher growth.

HINT: you can use pandas.pivot_table() to "flatten" the table (LARGE and LARGEST growth_7d as columns)

In [76]:
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS
# LARGEST_STOCKS = ['AAPL']

In [77]:
import time


def get_growth_7d(tickers):
  stocks_df = pd.DataFrame({'A' : []})
  for i, ticker in enumerate(tickers):
    print(i, ticker)

    # Work with stock prices
    history_prices = yf.download(tickers=ticker,
                                start='2013-12-19',
                                end='2023-12-31',
                                interval="1d")
    history_prices['Ticker'] = ticker
    history_prices['Year']= history_prices.index.year
    history_prices['Month'] = history_prices.index.month
    history_prices['Weekday'] = history_prices.index.weekday
    history_prices['Date'] = history_prices.index.date

    history_prices['growth_7d'] = history_prices['Adj Close'] / history_prices['Adj Close'].shift(7)

    # sleep 1 sec between downloads - not to overload the API server
    time.sleep(1)


    if stocks_df.empty:
      stocks_df = history_prices
    else:
      stocks_df = pd.concat([stocks_df, history_prices], ignore_index=True)
  return stocks_df

In [78]:
largest_stocks_df = get_growth_7d(LARGEST_STOCKS)

0 MSFT


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 AAPL



[*********************100%%**********************]  1 of 1 completed

2 GOOG


...

30 ITC.NS


[*********************100%%**********************]  1 of 1 completed


31 HINDUNILVR.NS


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

32 LT.NS


In [79]:
NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

In [80]:
large_stocks_df = get_growth_7d(LARGE_STOCKS)

[*********************100%%**********************]  1 of 1 completed

0 PRX.AS



[*********************100%%**********************]  1 of 1 completed

1 CDI.PA


2 AIR.PA


...

31 NTPC.NS


[*********************100%%**********************]  1 of 1 completed


32 KOTAKBANK.NS


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

33 TITAN.NS


In [81]:
import datetime

largest_stocks_df = largest_stocks_df[largest_stocks_df['Date'] >= datetime.date(2014, 1, 1)]
large_stocks_df = large_stocks_df[large_stocks_df['Date'] >= datetime.date(2014, 1, 1)]

In [88]:
largest_stocks_df_grouped = largest_stocks_df.groupby(by=['Date']).agg({'growth_7d': 'mean'}).reset_index()

In [89]:
large_stocks_df_grouped = large_stocks_df.groupby(by=['Date']).agg({'growth_7d': 'mean'}).reset_index()

In [90]:
large_stocks_df_grouped['ticker_category'] = 'LARGE'
largest_stocks_df_grouped['ticker_category'] = 'LARGEST'

In [93]:
all_stocks_df = pd.merge(large_stocks_df_grouped, largest_stocks_df_grouped, how='inner', on='Date')

In [94]:
all_stocks_df

,Date,growth_7d_x,ticker_category_x,growth_7d_y,ticker_category_y
0,2014-01-01,1.011684,LARGE,1.011797,LARGEST
1,2014-01-02,1.000970,LARGE,1.005147,LARGEST
2,2014-01-03,0.999534,LARGE,0.997668,LARGEST
3,2014-01-06,0.995983,LARGE,0.994203,LARGEST
4,2014-01-07,0.991592,LARGE,0.992509,LARGEST
...,...,...,...,...,...
2590,2023-12-22,1.003014,LARGE,1.013788,LARGEST
2591,2023-12-26,1.013505,LARGE,1.014900,LARGEST
2592,2023-12-27,1.002167,LARGE,1.014155,LARGEST
2593,2023-12-28,0.998583,LARGE,1.008533,LARGEST


In [95]:
all_stocks_df[all_stocks_df['growth_7d_x'] >= all_stocks_df['growth_7d_y']]

,Date,growth_7d_x,ticker_category_x,growth_7d_y,ticker_category_y
2,2014-01-03,0.999534,LARGE,0.997668,LARGEST
3,2014-01-06,0.995983,LARGE,0.994203,LARGEST
5,2014-01-08,0.988792,LARGE,0.987730,LARGEST
6,2014-01-09,0.990017,LARGE,0.982162,LARGEST
7,2014-01-10,0.987625,LARGE,0.983251,LARGEST
...,...,...,...,...,...
2577,2023-12-05,1.037530,LARGE,1.013552,LARGEST
2578,2023-12-06,1.036610,LARGE,1.015006,LARGEST
2579,2023-12-07,1.028125,LARGE,1.021288,LARGEST
2580,2023-12-08,1.030491,LARGE,1.025336,LARGEST


In [96]:
1231 / 2595

0.474373795761079

# Question 4: Trying Another Technical Indicators strategy
**What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?**

First, run the entire Colab to obtain the full DataFrame of data (after [Code Snippet 9]), and truncate it to the last full 10 years of data (2014-01-01 to 2023-12-31). If you encounter any difficulties running the Colab - you can download it using this link.

Let's assume you've learned about the awesome CCI indicator (Commodity Channel Index), and decided to use only it for your operations.

You defined the "defensive" value of a high threshould of 200, and you trade only on Fridays (Date.dt.dayofweek()==4).

That is, every time you see that CCI is >200 for any stock (out of those 33), you'll invest $1000 (each record when CCI>200) at Adj.Close price and hold it for 1 week (5 trading days) in order to sell at the Adj. Close price.

What's the expected gross profit (no fees) that you get in THOUSANDS $ (closest integer value) over many operations in 10 years? One operation calculations: if you invested $1000 and received $1010 in 5 days - you add $10 to gross profit, if you received $980 - add -$20 to gross profit. You need to sum these results over all trades (460 times in 10 years).

Additional:

Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")
are you still profitable on those trades?

In [104]:
stocks_df_combined = pd.read_parquet(f'stocks_df_combined_2024_05_09.parquet.brotli')

In [112]:
stocks_df_combined = stocks_df_combined[(stocks_df_combined['Date'] >= '2014-01-01') & (stocks_df_combined['Date'] <= '2023-12-31')]

In [113]:
for col in stocks_df_combined.columns:
  print(col)

Open
High
Low
Close
Adj Close_x
Volume
Ticker
Year
Month
Weekday
Date
growth_1d
growth_3d
growth_7d
growth_30d
growth_90d
growth_365d
growth_future_5d
SMA10
SMA20
growing_moving_average
high_minus_low_relative
volatility
is_positive_growth_5d_future
ticker_type
index_x
adx
adxr
apo
aroon_1
aroon_2
aroonosc
bop
cci
cmo
dx
macd
macdsignal
macdhist
macd_ext
macdsignal_ext
macdhist_ext
macd_fix
macdsignal_fix
macdhist_fix
mfi
minus_di
mom
plus_di
dm
ppo
roc
rocp
rocr
rocr100
rsi
slowk
slowd
fastk
fastd
fastk_rsi
fastd_rsi
trix
ultosc
willr
index_y
ad
adosc
obv
atr
natr
ht_dcperiod
ht_dcphase
ht_phasor_inphase
ht_phasor_quadrature
ht_sine_sine
ht_sine_leadsine
ht_trendmod
avgprice
medprice
typprice
wclprice
index
cdl2crows
cdl3blackrows
cdl3inside
cdl3linestrike
cdl3outside
cdl3starsinsouth
cdl3whitesoldiers
cdlabandonedbaby
cdladvancedblock
cdlbelthold
cdlbreakaway
cdlclosingmarubozu
cdlconcealbabyswall
cdlcounterattack
cdldarkcloudcover
cdldoji
cdldojistar
cdldragonflydoji
cdlengulfing
cd

In [119]:
stocks_combined_df_cci = stocks_df_combined[(stocks_df_combined['cci'] > 200) & (stocks_df_combined.Date.dt.dayofweek==4)]

In [120]:
stocks_combined_df_cci[['Date', 'Ticker', 'cci', 'Adj Close_x', 'growth_future_5d']]

,Date,Ticker,cci,Adj Close_x,growth_future_5d
7113,2014-05-30,MSFT,203.518798,34.912762,1.013190
7181,2014-09-05,MSFT,257.013330,39.395607,1.017207
7340,2015-04-24,MSFT,340.009871,41.630753,1.016503
7452,2015-10-02,MSFT,206.125828,40.151119,1.033794
7467,2015-10-23,MSFT,431.107024,46.583050,0.995650
...,...,...,...,...,...
5072,2022-11-25,LT.NS,210.797547,2058.108887,1.011029
5219,2023-06-30,LT.NS,219.106421,2469.979980,0.989417
5234,2023-07-21,LT.NS,359.236523,2580.430908,1.024476
5268,2023-09-08,LT.NS,303.781710,2901.600098,1.002395


In [121]:
stocks_combined_df_cci['profit'] = (stocks_combined_df_cci.growth_future_5d - 1) * 1000

<ipython-input-121-d1635f1795ae>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks_combined_df_cci['profit'] = (stocks_combined_df_cci.growth_future_5d - 1) * 1000


In [124]:
stocks_combined_df_cci.head()

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d,profit
7113,40.450001,40.970001,40.250000,40.939999,34.912762,34567600.0,MSFT,2014,2014-05-01,4,...,1.005607,1.021664,0.995813,NaN,NaN,NaN,NaN,NaN,NaN,13.190094
7181,45.110001,45.930000,45.110001,45.910000,39.395607,36939400.0,MSFT,2014,2014-09-01,4,...,0.941627,0.925124,0.914716,NaN,NaN,NaN,NaN,NaN,NaN,17.207462
7340,45.660000,48.139999,45.650002,47.869999,41.630753,130933700.0,MSFT,2015,2015-04-01,4,...,1.143658,1.055457,0.614516,0.978035,0.982994,1.037625,0.939362,0.933108,NaN,16.503238
7452,44.270000,45.570000,43.919998,45.570000,40.151119,41839000.0,MSFT,2015,2015-10-01,4,...,1.032390,0.775540,0.441114,0.998922,1.002560,1.009139,1.034930,0.909566,0.632660,33.794235
7467,52.299999,54.070000,52.250000,52.869999,46.583050,135227100.0,MSFT,2015,2015-10-01,4,...,0.996884,0.751370,0.435520,1.009025,1.026100,1.051840,1.200679,0.957738,0.771437,-4.350255


In [123]:
stocks_combined_df_cci['profit'].sum()

1048.2595204098575